In [3]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("mteb/tweet_sentiment_extraction")
df = pd.DataFrame(dataset['train'])


In [5]:
df.shape

(26732, 4)

In [9]:
df.head()

,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative


**Tokenizing **

In [10]:
from transformers import GPT2Tokenizer


In [13]:
# Loading the dataset to train our model
dataset = load_dataset("mteb/tweet_sentiment_extraction")


In [14]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [15]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/26732 [00:00<?, ? examples/s]

Map:   0%|          | 0/3432 [00:00<?, ? examples/s]

In [17]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 26732
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 3432
    })
})


**Spliting Tokenized Dataset into Train and Test to evaluate our fine tuned model**

In [18]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

**Loading our base model**

In [19]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00


In [29]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

**Tuning Using the Trainer Method**

In [27]:
# Completely disable W&B
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   output_dir="test_trainer",
   #evaluation_strategy="epoch",
   per_device_train_batch_size=1,  # Reduce batch size here
   per_device_eval_batch_size=1,    # Optionally, reduce for evaluation as well
   gradient_accumulation_steps=4
   )


trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,

)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,0.918400


TrainOutput(global_step=750, training_loss=0.7836607259114583, metrics={'train_runtime': 832.3292, 'train_samples_per_second': 3.604, 'train_steps_per_second': 0.901, 'total_flos': 1567794659328000.0, 'train_loss': 0.7836607259114583, 'epoch': 3.0})

# Evaluating the Fine Tuned Model

In [30]:
import evaluate

trainer.evaluate()


{'eval_loss': 0.8152584433555603,
 'eval_accuracy': 0.744,
 'eval_runtime': 79.0035,
 'eval_samples_per_second': 12.658,
 'eval_steps_per_second': 12.658,
 'epoch': 3.0}

In [32]:
import torch
from transformers import GPT2LMHeadModel

# Make sure tokenizer uses a pad token (GPT-2 has none by default)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id

model = GPT2LMHeadModel.from_pretrained("gpt2") # Change model class

model.eval()
model.config.use_cache = True  # faster at inference

prompt = "what interview! leave me alone"
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=60,
        temperature=0.8,     # higher = more random; set to 0 for greedy
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

what interview! leave me alone)
What is the key to keeping a positive attitude? I am looking for those who have found this guide helpful. Let's get started: 1. How can you help in your role as an individual that might be challenging but still rewarding when it comes to managing and maintaining relationships, friendships or community


**Testing my fine TUned Model**

In [33]:
trainer.save_model("test_trainer")         # saves model + config
tokenizer.save_pretrained("test_trainer")  # saves tokenizer

('test_trainer/tokenizer_config.json',
 'test_trainer/special_tokens_map.json',
 'test_trainer/vocab.json',
 'test_trainer/merges.txt',
 'test_trainer/added_tokens.json')

In [34]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

ckpt = "test_trainer"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(ckpt)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = True
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

prompt = "what interview! leave me alone"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    out_ids = model.generate(
        **inputs,
        max_new_tokens=60,
        temperature=0.8,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(out_ids[0], skip_special_tokens=True))

what interview! leave me alone!!!!!!! Leave me alone!!!!!
(I would be very angry!My anger!
willFUL!Mydamage.mydestest !,most ,worst -severe.iheart-diserending:includingpoisoned anddesperationofall types ofanimalislandin
